# Netflixも使っている！Contextual Banditアルゴリズムを徹底解説！(Part 1)

In [1]:
import numpy as np
import pandas as pd

from pandas import DataFrame
from plotly.offline import iplot, plot

In [2]:
from pymab.bandit import BernoulliBandit, GaussianBandit
from pymab.policy import EpsilonGreedy, GaussianThompsonSampling, LinUCB, LinTS, LogisticTS
from pymab.sim import BanditSimulator

### 実験1（誤差項が正規分布）
- 配信回数20,000回の中で作品の視聴時間など（目的変数が連続）を最大化するという状況設定. <br>
- 各アルゴリズムのパラメータ推定値は300回の試行につき1回更新できるものとする.<br>
- LinTSは$\tilde{\theta}$のサンプリングが速度的なボトルネックであるため, 15回の試行につき1回サンプリング.
- 100回のシミュレーションにおける平均累積リグレットで性能比較. <br>

#### 比較アルゴリズム
| アルゴリズム | ハイパーパラメータ |
|:-----------|:------------|
|ε-greedy<br>(比較用)|$\epsilon = 0.1$|
|LinTS|$\sigma_0 = 0.1$|
|LinTS |$\sigma_0 = 1$ |
|LinUCB|$\alpha = 0.1$ |
|LinUCB|$\alpha = 1$ | |

#### アームの作り方
- 5種類のアーム（Artworkの選択肢）を用意.
- 各アームは20次元の真の線形パラメータ$\theta_a$を標準正規分布から独立に得る.

#### 特徴量の作り方
- 文脈は全て0 or 1の離散値とし, 各ユーザーについて20次元の文脈が観測される. (ダミー変数)

In [3]:
# ガウスバンディット.
n_arms, n_features = 5, 20
gb = GaussianBandit(n_arms=n_arms, n_features=n_features, noise=0.1, contextual=True)

# シミュレーション1に用いるアルゴリズムの指定.
pols1 =  [EpsilonGreedy(n_arms=n_arms, epsilon=0.1, batch_size=500),
          LinTS(n_arms=n_arms, n_features=n_features, sigma=0.1, sample_batch=15, batch_size=500),
          LinTS(n_arms=n_arms, n_features=n_features, sigma=1, sample_batch=15, batch_size=500),
          LinUCB(n_arms=n_arms, n_features=n_features, alpha=0.1, batch_size=500),
          LinUCB(n_arms=n_arms, n_features=n_features, alpha=1, batch_size=500)]

## Run Simulation

In [4]:
bs1 = BanditSimulator(policy_list=pols1, bandit=gb, num_sims=100, n_rounds=20000, contextual=True)

In [ ]:
bs1.run_sim()

Avg Elapsed Time(20000 iter) EpsilonGreedy(ε=0.1) : 0.013s
Avg Elapsed Time(20000 iter) LinTS(σ=0.1) : 0.056s


## Plot Result

In [ ]:
rewards_plot1, regret_plot1, bingo_plot1 = bs1.plots()

In [ ]:
iplot(rewards_plot1)
iplot(regret_plot1)
iplot(bingo_plot1)

### 実験2（報酬が2値変数）
- 配信回数20,000回の中で作品の視聴回数など（目的変数が離散）を最大化するという状況設定. <br>
- そのほかの設定は実験1と同じ.

#### 比較アルゴリズム
| アルゴリズム | ハイパーパラメータ |
|:-----------|:------------|
|ε-greedy<br>(比較用)|$\epsilon = 0.1$|
|LinUCB|$\alpha = 1$ |
|LinTS |$\sigma_0^2 = 1$ |
|LogisticTS|$\sigma_0^2 = 1$|

In [ ]:
# ベルヌーイバンディット.
bb = BernoulliBandit(n_arms=n_arms, n_features=n_features, noise=0.1, contextual=True)

# シミュレーション1に用いるアルゴリズムの指定.
pols2 =  [EpsilonGreedy(n_arms=n_arms, epsilon=0.1, batch_size=500),
          LinTS(n_arms=n_arms, n_features=n_features, sigma=1, sample_batch=15, batch_size=500),
          LinUCB(n_arms=n_arms, n_features=n_features, alpha=1, batch_size=500),
          LogisticTS(n_arms=n_arms, n_features=n_features, sigma=1, n_iter=2, sample_batch=15, batch_size=500)]

## Run Simulation

In [ ]:
bs2 = BanditSimulator(policy_list=pols2, bandit=bb, num_sims=100, n_rounds=20000, contextual=True)

In [ ]:
bs2.run_sim()

## Plot Result

In [ ]:
rewards_plot2, regret_plot2, bingo_plot2 = bs2.plots()

In [ ]:
iplot(rewards_plot2)
iplot(regret_plot2)
iplot(bingo_plot2)

## おまけ（本当にパラメータ推定できてる？）

#### 設定
- LinUCBを用いて, 線形モデルのパラメータをバッチリ推定しているのか試す.
- 平均0, 分散3.0の正規分布から8次元の$\theta$をサンプルする.
- アームの数は3種類. つまり, $8 \times 3 = 24$次元のパラメータを当てに行く.
- 10,000回の試行を経た後のLinUCBにおけるパラメータ推定値$\hat{\theta}$を見てみる.

In [23]:
# ガウスバンディット
n_arms, n_features = 5, 10
bandit = GaussianBandit(n_arms=n_arms, n_features=n_features, noise=1.0, contextual=True)


# データ生成
n_rounds = 10000
x = np.concatenate([np.expand_dims(np.random.randint(2, size=bandit.n_features), axis=0)
                    for i in np.arange(n_rounds)], axis=0)

In [24]:
# 真の線形パラメータ
bandit.params

array([[ 2.14548772, -0.54070832, -1.02230683,  1.64420467, -0.60341971],
       [ 0.50398959,  0.05339814, -0.5455923 , -0.0845019 , -0.08370617],
       [ 0.84175091, -1.14463286,  0.44554638,  0.75985572,  0.74978089],
       [-0.35666543, -0.71341971, -0.5889453 , -0.26778343,  0.24631797],
       [-0.97670393, -1.08069665,  0.96559637, -0.19370865,  0.4815092 ],
       [-1.01468062,  0.13943516,  2.67411566,  1.3232494 ,  0.06461185],
       [-0.18691928, -1.07666881, -0.67688435, -1.19585094,  0.68683834],
       [-1.47668816, -0.42535632, -0.35100311,  0.31708501,  1.14631281],
       [-1.37337703, -1.61982118, -0.06913663, -1.70530737,  0.16205655],
       [ 1.38961793, -2.56092716,  0.300219  ,  1.06999454, -1.98742231]])

## Run

In [25]:
# LinUCBによるパラメータ推定
player = LinUCB(n_arms=bandit.n_arms, n_features=bandit.n_features)

for t in np.arange(n_rounds):

    chosen_arm = player.select_arm(x[t, :])
    reward = bandit.pull(x=x[t, :], chosen_arm=chosen_arm)
    
    player.update(x[t, :], chosen_arm, reward)

In [26]:
# よく推定できている
player.theta_hat

array([[ 2.32764798, -0.39128711, -1.06747483,  1.61451971, -0.54931816],
       [ 0.47497562, -1.33609567, -0.49102157, -0.09943008, -0.14097625],
       [ 0.85237526, -0.39128711,  0.3302959 ,  0.69117123,  0.83292508],
       [-0.35917316, -0.91270767, -0.52674588, -0.20917378,  0.25397878],
       [-1.10048642, -0.94480856,  0.96363275, -0.15214091,  0.48342883],
       [-0.86011254,  0.        ,  2.70826309,  1.37237199,  0.08913352],
       [-0.28692172, -1.42201144, -0.67435329, -1.2678292 ,  0.65717052],
       [-1.58805512, -0.39128711, -0.43526705,  0.29037523,  1.08852971],
       [-1.47565586, -1.42201144, -0.07098755, -1.67978241,  0.14819927],
       [ 1.35889452, -1.81329855,  0.35794833,  1.08724568, -1.94262103]])